In [ ]:
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 1000)
INPUT = '/kaggle/input'
DATA = f'{INPUT}/shopee-product-matching'
sys.path.append(f"{INPUT}/symspellpy/symspellpy-6.7.0")
import pkg_resources
from symspellpy import SymSpell, Verbosity

# Spelling correction one word at a time

In [ ]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# lookup suggestions for single-word input strings
input_term = "memebers"  # misspelling of "members"
# max edit distance per lookup
# (max_edit_distance_lookup <= max_dictionary_edit_distance)
suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST,
                               max_edit_distance=2)
# display suggestion term, term frequency, and edit distance
for suggestion in suggestions:
    print(suggestion)

In [ ]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# lookup suggestions for multi-word input strings (supports compound
# splitting & merging)
input_term = "whereis th elove hehad dated forImuch of thepast who couqdn'tread in sixtgrade and ins pired him"
# max edit distance per lookup (per single word, not per whole input string)
suggestions = sym_spell.lookup_compound(input_term, max_edit_distance=2)
# display suggestion term, edit distance, and term frequency
for suggestion in suggestions:
    print(suggestion)

In [ ]:
train = pd.read_csv(f"{DATA}/train.csv", engine="c", low_memory=False)
train.info()

In [ ]:
en_words = set()
with open(f"{INPUT}/symspellpy/symspellpy-6.7.0/symspellpy/frequency_dictionary_en_82_765.txt") as lines:
    for line in lines:
        tokens = line.split()
        en_words.add(" ".join(tokens[:-1]))
print(f"{len(en_words)} en_words={list(en_words)[:100]}")

In [ ]:
def autocorrect(row):
    s = row["title"]
    tokens = s.split()
    res = []
    for t in tokens:
        r = t
        if t.lower() not in en_words:
            # TODO regex ignore punctuation
            suggestions = sym_spell.lookup(t, Verbosity.TOP, max_edit_distance=1, include_unknown=True, ignore_token=r"[\d\&]+")
            r = suggestions[0].term
        res.append(r)
    return " ".join(res)


In [ ]:
%%time
train["title_p"] = train.apply(autocorrect, axis=1)

In [ ]:
cols = ["title_p", "title"]
train[cols].head(50)

Problems
- `Double tape`: `double rape`


In [ ]:
#"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE".split()
len(en_words)